**When will the United States reach herd immunity (>230M) for COVID-19?**

Goal: Preict the date of when the first reliable media report is published that states that >230M of the US population (~70%) have either received a SARS-CoV-2 vaccine or has been previously been infected by the virus.

Factors:
* COVID case growth rates
* Vaccines timeline (getting EUA, starting distribution, achieving sufficient scale)
* Delay between achievement and the media article

https://www.metaculus.com/questions/5773/united-states-covid-herd-immunity-230m-date/

In [26]:
import math
import random

import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime, timedelta


# Model variables
GROWTH_DERIVATIVE_FACTOR_LOW = 0.5
GROWTH_DERIVATIVE_FACTOR_HIGH = 3.0

MONTHLY_GROWTH_MODIFIER_LOW = 0.5
MONTHLY_GROWTH_MODIFIER_HIGH = 3.0

DAYS_UNTIL_PEAK_GROWTH_LOW = -10
DAYS_UNTIL_PEAK_GROWTH_HIGH = 60

VACCINE_MODIFIER_LOW = 0.5
VACCINE_MODIFIER_HIGH = 1.5

RANDOM_CASE_FLUCTUATION_LOW = -2000
RANDOM_CASE_FLUCTUATION_HIGH = 4000

DAILY_FOURTH_WAVE_CHANCE = 0.6/365
FOURTH_WAVE_LENGTH_DAYS_LOW = 30
FOURTH_WAVE_LENGTH_DAYS_HIGH = 120

VACCINE_DELAY_DAYS_LOW = 7
VACCINE_DELAY_DAYS_HIGH = 28

NEWS_ARTICLE_DELAY_DAYS_LOW = -10 # Earlier than expected based on estimated cases rather than reported cases
NEWS_ARTICLE_DELAY_DAYS_HIGH = 30

million = 1000000

MONTHLY_VACCINATION_DAILY_AMOUNTS = [np.round(10 * million * (1/30) * (1/12), 0),
                                     np.round(10 * million * (1/30) * (1/12), 0),
                                     np.round(50 * million * (1/30) * (1/12), 0),
                                     np.round(60 * million * (1/30) * (1/12), 0),
                                     np.round(70 * million * (1/30) * (1/12), 0),
                                     np.round(80 * million * (1/30) * (1/12), 0),
                                     np.round(100 * million * (1/30) * (1/12), 0),
                                     np.round(150 * million * (1/30) * (1/12), 0),
                                     np.round(200 * million * (1/30) * (1/12), 0),
                                     np.round(250 * million * (1/30) * (1/12), 0),
                                     np.round(300 * million * (1/30) * (1/12), 0),
                                     np.round(350 * million * (1/30) * (1/12), 0),
                                     np.round(400 * million * (1/30) * (1/12), 0)]

# Simulation parameters
N_SCENARIOS = 1000
SIM_START_DATE = '2020-11-28'
SIM_END_DATE = '2022-12-21'
CURRENT_COVID_CASES = 13.6 * million
CURRENT_COVID_GROWTH = 158000
CURRENT_COVID_DERIVATIVE = 2000
Q4_2020_START_COVID = 6.03 * million
TARGET_POP = 230 * million
US_POP = 328.2 * million

In [27]:
def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def get_metaculus_range(rangex, minx=None, maxx=None):
    lower25, mean, upper75 = np.percentile(rangex, [25, 50, 75])
    results = {}
    results['lower25'] = lower25
    results['mean'] = mean
    results['upper75'] = upper75
    if minx:
        results['minx'] = print_pct(sum([x < minx for x in rangex]) / len(rangex))
    if maxx:
        results['maxx'] = print_pct(sum([x > maxx for x in rangex]) / len(rangex))
    return results
    

def normal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        mu = (high + low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (high - mu) / normed_sigma
        return np.random.normal(mu, sigma)

    
def lognormal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        log_low = np.log(low)
        log_high = np.log(high)
        mu = (log_high + log_low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (log_high - mu) / normed_sigma
        return np.random.lognormal(mu, sigma)

In [28]:
days_until_sim_end_date = (datetime.strptime(SIM_END_DATE, '%Y-%m-%d') - datetime.strptime(SIM_START_DATE, '%Y-%m-%d')).days + 1

dec_7_2020_covid_collector = []
y2020_covid_collector = []
q1_2021_covid_collector = []
q2_2021_covid_collector = []
q3_2021_covid_collector = []
q4_2021_covid_collector = []
q1_2021_vaccinated_collector = []
q2_2021_vaccinated_collector = []
q3_2021_vaccinated_collector = []
q4_2021_vaccinated_collector = []
q1_2021_target_collector = []
q2_2021_target_collector = []
q3_2021_target_collector = []
q4_2021_target_collector = []
target_date_collector = []


for n in range(N_SCENARIOS):
    done = False
    fourth_wave = False
    today = datetime.today()
    
    covid = CURRENT_COVID_CASES
    covid_growth = CURRENT_COVID_GROWTH
    
    growth_derivative_factor = np.round(normal_sample(GROWTH_DERIVATIVE_FACTOR_LOW, GROWTH_DERIVATIVE_FACTOR_HIGH, 0.8) + 1, 1)
    if growth_derivative_factor < 0.1:
        growth_derivative_factor = 0.1
    
    monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
    if monthly_growth_modifier < 0.5:
        monthly_growth_modifier = 0.5
     
    peak_growth_days = np.round(normal_sample(DAYS_UNTIL_PEAK_GROWTH_LOW, DAYS_UNTIL_PEAK_GROWTH_HIGH, 0.8), 0)
    if peak_growth_days < 0:
        peak_growth_days = 0
    peak_growth_date = (today + timedelta(days=peak_growth_days)).date()
    
    fourth_wave_days_length = np.round(lognormal_sample(FOURTH_WAVE_LENGTH_DAYS_LOW, FOURTH_WAVE_LENGTH_DAYS_HIGH, 0.8), 0)
    if fourth_wave_days_length < 1:
        fourth_wave_days_length = 1
        
    vaccine_modifier = np.round(normal_sample(VACCINE_MODIFIER_LOW, VACCINE_MODIFIER_HIGH, 0.8), 1)
    if vaccine_modifier < 0.3:
        vaccine_modifier = 0.3
        
    vaccine_delay_days = np.round(normal_sample(VACCINE_DELAY_DAYS_LOW, VACCINE_DELAY_DAYS_HIGH, 0.8), 1)
        
    news_article_delay_days = np.round(normal_sample(NEWS_ARTICLE_DELAY_DAYS_LOW, NEWS_ARTICLE_DELAY_DAYS_HIGH, 0.8), 1)
    
    vaccine_rate = 0
    total_vaccinated = 0
    
    verbose = (n <= 4) or (n % 10 == 0)
    really_verbose = (n <= 4)
    
    if verbose:
        print('-')
        print('-')
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))
        print('Growth derivative factor: {}'.format(growth_derivative_factor))
        print('Vaccine modifier: {}'.format(vaccine_modifier))
        print('Peak growth on: {}'.format(peak_growth_date))
        print('Vaccine delay days: {}'.format(vaccine_delay_days))
        print('News article delay days: {}'.format(news_article_delay_days))

    for day in range(days_until_sim_end_date):
        date = str((today + timedelta(days = day)).date())
        if really_verbose:
            print('Day {}: {}'.format(day, date))
            
        # Grow vaccines
        days_since_vaccine_start = (today + timedelta(days = day) - datetime(2020, 12, 15) - timedelta(days = vaccine_delay_days)).days
        if days_since_vaccine_start >= 0:
            i = math.floor(days_since_vaccine_start / 30)
            if i >= len(MONTHLY_VACCINATION_DAILY_AMOUNTS):
                i = len(MONTHLY_VACCINATION_DAILY_AMOUNTS) - 1
            daily_vaccines = MONTHLY_VACCINATION_DAILY_AMOUNTS[i]
            old_vaccine_rate = vaccine_rate
            vaccine_rate = np.round(np.random.normal(daily_vaccines * vaccine_modifier,
                                                     daily_vaccines * vaccine_modifier), 0)
            if vaccine_rate < old_vaccine_rate:
                vaccine_rate = old_vaccine_rate
            total_vaccinated = np.round(total_vaccinated + vaccine_rate, 0)
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
            
        # Grow COVID
        if day > peak_growth_days and (not fourth_wave or day > fourth_wave_end):
            covid_growth = covid_growth - ((CURRENT_COVID_DERIVATIVE / growth_derivative_factor) * (1 / monthly_growth_modifier))
        else:
            covid_growth = covid_growth + ((CURRENT_COVID_DERIVATIVE * growth_derivative_factor) * monthly_growth_modifier)
        covid_growth = np.round(covid_growth, 0)
        old_covid_growth = covid_growth
        if old_covid_growth < 0:
            old_covid_growth = 0
            
        covid_growth -= np.round(vaccine_rate * 0.02, 0)
        if covid_growth < 0:
            covid_growth = 0
            
        random_case_fluctuation = np.round(normal_sample(RANDOM_CASE_FLUCTUATION_LOW, RANDOM_CASE_FLUCTUATION_HIGH, 0.8), 0)
        covid = covid + covid_growth + random_case_fluctuation
        current_pop = covid + total_vaccinated
        current_target = current_pop / US_POP * 100
        if current_pop >= TARGET_POP and not done:
            target_date_collector.append(day + news_article_delay_days)
            done = True
            
        if really_verbose:
            print('...Total COVID cases now {} (+{} {} -{} *{})'.format(covid,
                                                                        old_covid_growth,
                                                                        "%+d" % random_case_fluctuation,
                                                                        np.round(vaccine_rate * 0.02, 0),
                                                                        monthly_growth_modifier))
            print('...% of target: {}%/{}%'.format(np.round(current_target, 1),
                                                   np.round(TARGET_POP / US_POP * 100, 1)))
                
        if not fourth_wave and np.random.random() <= DAILY_FOURTH_WAVE_CHANCE:
            if really_verbose:
                print('...COVID FOURTH WAVE!!!')
            fourth_wave = True
            fourth_wave_end = day + fourth_wave_days_length
            
            
        # Record
        if date == '2020-12-07':
            dec7_covid = covid
            if verbose:
                print('...COVID by 7 Dec: {}'.format(dec7_covid))
                    
        if date == '2020-12-31':
            y2020_covid = covid
            if verbose:
                print('...Total 2020 COVID: {}'.format(covid))
                
        if date.split('-')[-1] == '01':
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-03-31':
            q1_2021_covid = covid
            q1_2021_vaccinated = total_vaccinated
            q1_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q1 2021 COVID: {}'.format(q1_2021_covid))
                print('...Q1 2021 Vaccinated: {}'.format(q1_2021_vaccinated))
                print('...Q1 2021 Target: {}'.format(q1_2021_target))

        if date == '2021-06-30':
            q2_2021_covid = covid
            q2_2021_vaccinated = total_vaccinated
            q2_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q2 2021 COVID: {}'.format(q2_2021_covid))
                print('...Q2 2021 Vaccinated: {}'.format(q2_2021_vaccinated))
                print('...Q2 2021 Target: {}'.format(q2_2021_target))
            
        if date == '2021-09-30':
            q3_2021_covid = covid
            q3_2021_vaccinated = total_vaccinated
            q3_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q3 2021 COVID: {}'.format(q3_2021_covid))
                print('...Q3 2021 Vaccinated: {}'.format(q3_2021_vaccinated))
                print('...Q3 2021 Target: {}'.format(q3_2021_target))
                
        if date == '2021-12-31':
            q4_2021_covid = covid
            q4_2021_vaccinated = total_vaccinated
            q4_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q4 2021 COVID: {}'.format(q4_2021_covid))
                print('...Q4 2021 Vaccinated: {}'.format(q4_2021_vaccinated))
                print('...Q4 2021 Target: {}'.format(q4_2021_target))
   
    if not done:
        target_date_collector.append(1000)

    dec_7_2020_covid_collector.append(dec7_covid)
    y2020_covid_collector.append(y2020_covid)
    q1_2021_covid_collector.append(q1_2021_covid)
    q2_2021_covid_collector.append(q2_2021_covid)
    q3_2021_covid_collector.append(q3_2021_covid)
    q4_2021_covid_collector.append(q4_2021_covid)
    q1_2021_vaccinated_collector.append(q1_2021_vaccinated)
    q2_2021_vaccinated_collector.append(q2_2021_vaccinated)
    q3_2021_vaccinated_collector.append(q3_2021_vaccinated)
    q4_2021_vaccinated_collector.append(q4_2021_vaccinated)
    q1_2021_target_collector.append(q1_2021_target)
    q2_2021_target_collector.append(q2_2021_target)
    q3_2021_target_collector.append(q3_2021_target)
    q4_2021_target_collector.append(q4_2021_target)

-
-
## SCENARIO 1/1000 ##
Growth derivative factor: 0.6
Vaccine modifier: 0.3
Peak growth on: 2020-12-27
Vaccine delay days: 18.9
News article delay days: 25.4
Day 0: 2020-12-03
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 13758988.0 (+159080.0 -92 -0.0 *0.9)
...% of target: 4.2%/70.1%
Day 1: 2020-12-04
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 13919946.0 (+160160.0 +798 -0.0 *0.9)
...% of target: 4.2%/70.1%
Day 2: 2020-12-05
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14076967.0 (+161240.0 -4219 -0.0 *0.9)
...% of target: 4.3%/70.1%
Day 3: 2020-12-06
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14235738.0 (+162320.0 -3549 -0.0 *0.9)
...% of target: 4.3%/70.1%
Day 4: 2020-12-07
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14397530.0 (+163400.0 -1608 -0.0 *0.9)
...% of target:

Day 252: 2021-08-12
...Total vaccinated now 29783606.0 (+408664.0)
...Total vaccinated now 29783606.0 (+408664.0)
...Total COVID cases now 28239187.0 (+3240.0 -3692 -8173.0 *2.7)
...% of target: 17.7%/70.1%
Day 253: 2021-08-13
...Total vaccinated now 30192270.0 (+408664.0)
...Total vaccinated now 30192270.0 (+408664.0)
...Total COVID cases now 28236010.0 (+3240.0 -3177 -8173.0 *2.7)
...% of target: 17.8%/70.1%
Day 254: 2021-08-14
...Total vaccinated now 30600934.0 (+408664.0)
...Total vaccinated now 30600934.0 (+408664.0)
...Total COVID cases now 28237884.0 (+3240.0 +1874 -8173.0 *2.7)
...% of target: 17.9%/70.1%
Day 255: 2021-08-15
...Total vaccinated now 31009598.0 (+408664.0)
...Total vaccinated now 31009598.0 (+408664.0)
...Total COVID cases now 28239843.0 (+3240.0 +1959 -8173.0 *2.7)
...% of target: 18.1%/70.1%
Day 256: 2021-08-16
...Total vaccinated now 31418262.0 (+408664.0)
...Total vaccinated now 31418262.0 (+408664.0)
...Total COVID cases now 28240880.0 (+3240.0 +1037 -8173.0

...% of target: 67.2%/70.1%
Day 451: 2022-02-27
...Total vaccinated now 193084278.0 (+1035687.0)
...Total vaccinated now 193084278.0 (+1035687.0)
...Total COVID cases now 28471550.0 (+0 -211 -20714.0 *3.8)
...% of target: 67.5%/70.1%
Day 452: 2022-02-28
...Total vaccinated now 194119965.0 (+1035687.0)
...Total vaccinated now 194119965.0 (+1035687.0)
...Total COVID cases now 28470133.0 (+0 -1417 -20714.0 *3.8)
...% of target: 67.8%/70.1%
Day 453: 2022-03-01
...Total vaccinated now 195155652.0 (+1035687.0)
...Total vaccinated now 195155652.0 (+1035687.0)
...Total COVID cases now 28468311.0 (+0 -1822 -20714.0 *3.8)
...% of target: 68.1%/70.1%
Day 454: 2022-03-02
...Total vaccinated now 196191339.0 (+1035687.0)
...Total vaccinated now 196191339.0 (+1035687.0)
...Total COVID cases now 28472597.0 (+0 +4286 -20714.0 *3.6)
...% of target: 68.5%/70.1%
Day 455: 2022-03-03
...Total vaccinated now 197227026.0 (+1035687.0)
...Total vaccinated now 197227026.0 (+1035687.0)
...Total COVID cases now 28

...Total COVID cases now 28677320.0 (+0 -1376 -22955.0 *1.9)
...% of target: 139.1%/70.1%
Day 658: 2022-09-22
...Total vaccinated now 429127925.0 (+1147757.0)
...Total vaccinated now 429127925.0 (+1147757.0)
...Total COVID cases now 28676575.0 (+0 -745 -22955.0 *1.9)
...% of target: 139.5%/70.1%
Day 659: 2022-09-23
...Total vaccinated now 430275682.0 (+1147757.0)
...Total vaccinated now 430275682.0 (+1147757.0)
...Total COVID cases now 28679400.0 (+0 +2825 -22955.0 *1.9)
...% of target: 139.8%/70.1%
Day 660: 2022-09-24
...Total vaccinated now 431423439.0 (+1147757.0)
...Total vaccinated now 431423439.0 (+1147757.0)
...Total COVID cases now 28678480.0 (+0 -920 -22955.0 *1.9)
...% of target: 140.2%/70.1%
Day 661: 2022-09-25
...Total vaccinated now 432571196.0 (+1147757.0)
...Total vaccinated now 432571196.0 (+1147757.0)
...Total COVID cases now 28678587.0 (+0 +107 -22955.0 *1.9)
...% of target: 140.5%/70.1%
Day 662: 2022-09-26
...Total vaccinated now 433718953.0 (+1147757.0)
...Total vac

...Total COVID cases now 24808345.0 (+0 -914 -10372.0 *3.7)
...% of target: 14.8%/70.1%
Day 140: 2021-04-22
...Total vaccinated now 24222462.0 (+518582.0)
...Total vaccinated now 24222462.0 (+518582.0)
...Total COVID cases now 24810761.0 (+0 +2416 -10372.0 *3.7)
...% of target: 14.9%/70.1%
Day 141: 2021-04-23
...Total vaccinated now 24741044.0 (+518582.0)
...Total vaccinated now 24741044.0 (+518582.0)
...Total COVID cases now 24811650.0 (+0 +889 -10372.0 *3.7)
...% of target: 15.1%/70.1%
Day 142: 2021-04-24
...Total vaccinated now 25259626.0 (+518582.0)
...Total vaccinated now 25259626.0 (+518582.0)
...Total COVID cases now 24817749.0 (+0 +6099 -10372.0 *3.7)
...% of target: 15.3%/70.1%
Day 143: 2021-04-25
...Total vaccinated now 25778208.0 (+518582.0)
...Total vaccinated now 25778208.0 (+518582.0)
...Total COVID cases now 24816137.0 (+0 -1612 -10372.0 *3.7)
...% of target: 15.4%/70.1%
Day 144: 2021-04-26
...Total vaccinated now 26310420.0 (+532212.0)
...Total vaccinated now 26310420.0

...

-
-
## SCENARIO 311/1000 ##
Growth derivative factor: 3.6
Vaccine modifier: 0.9
Peak growth on: 2020-12-30
Vaccine delay days: 8.0
News article delay days: 3.5
...COVID by 7 Dec: 14589664.0
...Total 2020 COVID: 23764074.0
...Q1 2021 COVID: 56799935.0
...Q1 2021 Vaccinated: 18253592.0
...Q1 2021 Target: 22.9
...Q2 2021 COVID: 57786119.0
...Q2 2021 Vaccinated: 65849107.0
...Q2 2021 Target: 37.7
...Q3 2021 COVID: 57884561.0
...Q3 2021 Vaccinated: 160113249.0
...Q3 2021 Target: 66.4
...Q4 2021 COVID: 57983005.0
...Q4 2021 Vaccinated: 344597972.0
...Q4 2021 Target: 122.7
-
-
## SCENARIO 321/1000 ##
Growth derivative factor: 1.1
Vaccine modifier: 0.6
Peak growth on: 2021-01-07
Vaccine delay days: 24.3
News article delay days: 2.0
...COVID by 7 Dec: 14478130.0
...Total 2020 COVID: 20867994.0
...Q1 2021 COVID: 47699917.0
...Q1 2021 Vaccinated: 6034192.0
...Q1 2021 Target: 16.4
...Q2 2021 COVID: 50772097.0
...Q2 2021 Vaccinated: 38198752.0
...Q2 2021 Target: 27.1
...Q3 2021 COVID: 50852838.0
..

-
-
## SCENARIO 461/1000 ##
Growth derivative factor: 3.0
Vaccine modifier: 0.6
Peak growth on: 2021-01-04
Vaccine delay days: 4.8
News article delay days: 26.1
...COVID by 7 Dec: 14573561.0
...Total 2020 COVID: 23370992.0
...Q1 2021 COVID: 64579264.0
...Q1 2021 Vaccinated: 14137365.0
...Q1 2021 Target: 24.0
...Q2 2021 COVID: 68978760.0
...Q2 2021 Vaccinated: 45586601.0
...Q2 2021 Target: 34.9
...Q3 2021 COVID: 71114633.0
...Q3 2021 Vaccinated: 113884288.0
...Q3 2021 Target: 56.4
...Q4 2021 COVID: 71198107.0
...Q4 2021 Vaccinated: 257439891.0
...Q4 2021 Target: 100.1
-
-
## SCENARIO 471/1000 ##
Growth derivative factor: 3.0
Vaccine modifier: 0.9
Peak growth on: 2020-12-25
Vaccine delay days: 11.7
News article delay days: 4.6
...COVID by 7 Dec: 14764110.0
...Total 2020 COVID: 29138847.0
...Q1 2021 COVID: 159869680.0
...Q1 2021 Vaccinated: 18564590.0
...Q1 2021 Target: 54.4
...Q2 2021 COVID: 331631526.0
...Q2 2021 Vaccinated: 65117051.0
...Q2 2021 Target: 120.9
...Q3 2021 COVID: 39085164

-
-
## SCENARIO 611/1000 ##
Growth derivative factor: 2.8
Vaccine modifier: 1.1
Peak growth on: 2020-12-04
Vaccine delay days: 12.2
News article delay days: -8.7
...COVID by 7 Dec: 14526948.0
...Total 2020 COVID: 18956637.0
...Q1 2021 COVID: 26632898.0
...Q1 2021 Vaccinated: 20054599.0
...Q1 2021 Target: 14.2
...Q2 2021 COVID: 26746865.0
...Q2 2021 Vaccinated: 78915474.0
...Q2 2021 Target: 32.2
...Q3 2021 COVID: 26824878.0
...Q3 2021 Vaccinated: 193880305.0
...Q3 2021 Target: 67.2
...Q4 2021 COVID: 26920124.0
...Q4 2021 Vaccinated: 412418158.0
...Q4 2021 Target: 133.9
-
-
## SCENARIO 621/1000 ##
Growth derivative factor: 1.6
Vaccine modifier: 1.2
Peak growth on: 2020-12-20
Vaccine delay days: 8.4
News article delay days: 26.6
...COVID by 7 Dec: 14545435.0
...Total 2020 COVID: 21562045.0
...Q1 2021 COVID: 37218033.0
...Q1 2021 Vaccinated: 22617131.0
...Q1 2021 Target: 18.2
...Q2 2021 COVID: 37255014.0
...Q2 2021 Vaccinated: 92815581.0
...Q2 2021 Target: 39.6
...Q3 2021 COVID: 37352731.0

-
-
## SCENARIO 761/1000 ##
Growth derivative factor: 2.3
Vaccine modifier: 1.1
Peak growth on: 2020-12-24
Vaccine delay days: 0.3
News article delay days: -8.2
...COVID by 7 Dec: 14615305.0
...Total 2020 COVID: 23936129.0
...Q1 2021 COVID: 48362474.0
...Q1 2021 Vaccinated: 28126621.0
...Q1 2021 Target: 23.3
...Q2 2021 COVID: 48457870.0
...Q2 2021 Vaccinated: 91048639.0
...Q2 2021 Target: 42.5
...Q3 2021 COVID: 48537550.0
...Q3 2021 Vaccinated: 233105845.0
...Q3 2021 Target: 85.8
...Q4 2021 COVID: 48609535.0
...Q4 2021 Vaccinated: 493714181.0
...Q4 2021 Target: 165.2
-
-
## SCENARIO 771/1000 ##
Growth derivative factor: 2.8
Vaccine modifier: 1.7
Peak growth on: 2021-01-21
Vaccine delay days: 14.6
News article delay days: -6.3
...COVID by 7 Dec: 14609229.0
...Total 2020 COVID: 24280928.0
...Q1 2021 COVID: 120640714.0
...Q1 2021 Vaccinated: 38287062.0
...Q1 2021 Target: 48.4
...Q2 2021 COVID: 137600070.0
...Q2 2021 Vaccinated: 138955430.0
...Q2 2021 Target: 84.3
...Q3 2021 COVID: 1377070

-
-
## SCENARIO 911/1000 ##
Growth derivative factor: 3.8
Vaccine modifier: 1.5
Peak growth on: 2020-12-15
Vaccine delay days: 2.8
News article delay days: 8.6
...COVID by 7 Dec: 14737006.0
...Total 2020 COVID: 24916269.0
...Q1 2021 COVID: 50025939.0
...Q1 2021 Vaccinated: 25111624.0
...Q1 2021 Target: 22.9
...Q2 2021 COVID: 52481923.0
...Q2 2021 Vaccinated: 128857313.0
...Q2 2021 Target: 55.3
...Q3 2021 COVID: 52574139.0
...Q3 2021 Vaccinated: 344787911.0
...Q3 2021 Target: 121.1
...Q4 2021 COVID: 52696306.0
...Q4 2021 Vaccinated: 712559934.0
...Q4 2021 Target: 233.2
-
-
## SCENARIO 921/1000 ##
Growth derivative factor: 1.1
Vaccine modifier: 0.9
Peak growth on: 2021-01-30
Vaccine delay days: 27.6
News article delay days: -9.1
...COVID by 7 Dec: 14504442.0
...Total 2020 COVID: 21286140.0
...Q1 2021 COVID: 58827242.0
...Q1 2021 Vaccinated: 10446968.0
...Q1 2021 Target: 21.1
...Q2 2021 COVID: 62859044.0
...Q2 2021 Vaccinated: 58219449.0
...Q2 2021 Target: 36.9
...Q3 2021 COVID: 62940711.

## Total US COVID by 7 Dec

For calibration

In [29]:
get_metaculus_range(dec_7_2020_covid_collector)

{'lower25': 14494655.5, 'mean': 14568265.0, 'upper75': 14672449.25}

## By EOY 2020

In [30]:
get_metaculus_range(y2020_covid_collector)

{'lower25': 19554047.0, 'mean': 22368627.0, 'upper75': 25447940.5}

## By EOQ Q1 2021

In [31]:
get_metaculus_range(q1_2021_covid_collector)

{'lower25': 32297359.75, 'mean': 60386525.0, 'upper75': 94661679.0}

In [32]:
get_metaculus_range(q1_2021_vaccinated_collector)

{'lower25': 10260069.75, 'mean': 14921576.0, 'upper75': 20034881.0}

In [33]:
get_metaculus_range(q1_2021_target_collector)

{'lower25': 15.275000000000002, 'mean': 22.9, 'upper75': 33.4}

## By EOQ Q2 2021

In [34]:
get_metaculus_range(q2_2021_covid_collector)

{'lower25': 34430724.5, 'mean': 67987014.5, 'upper75': 127317404.25}

In [35]:
get_metaculus_range(q2_2021_vaccinated_collector)

{'lower25': 48221318.0, 'mean': 66667993.5, 'upper75': 85974089.5}

In [36]:
get_metaculus_range(q2_2021_target_collector)

{'lower25': 32.5, 'mean': 43.55, 'upper75': 60.525}

## By EOQ Q3 2021

In [37]:
get_metaculus_range(q3_2021_covid_collector)

{'lower25': 35172870.75, 'mean': 70080155.0, 'upper75': 130731298.25}

In [38]:
get_metaculus_range(q3_2021_vaccinated_collector)

{'lower25': 128483633.25, 'mean': 174875503.5, 'upper75': 224432846.5}

In [39]:
get_metaculus_range(q3_2021_target_collector)

{'lower25': 63.5, 'mean': 81.2, 'upper75': 102.45000000000002}

## By EOY 2021

In [40]:
get_metaculus_range(q4_2021_covid_collector)

{'lower25': 35312132.0, 'mean': 70409214.5, 'upper75': 130978917.25}

In [41]:
get_metaculus_range(q4_2021_vaccinated_collector)

{'lower25': 295537678.75, 'mean': 400633280.0, 'upper75': 517523586.0}

In [42]:
get_metaculus_range(q4_2021_target_collector)

{'lower25': 120.45, 'mean': 153.0, 'upper75': 188.4}

## Target Date

In [43]:
'target not exceeded by 2022-12-21 in {} of scenarios'.format(print_pct(sum([x == days_until_sim_end_date for x in target_date_collector]) / len(target_date_collector)))

'target not exceeded by 2022-12-21 in <1.0% of scenarios'

In [44]:
results = get_metaculus_range([x for x in target_date_collector if x < days_until_sim_end_date])
results['lower25'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['lower25'])).date())
results['mean'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['mean'])).date())
results['upper75'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['upper75'])).date())
results

{'lower25': '2021-08-02', 'mean': '2021-09-13', 'upper75': '2021-10-21'}